In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import dateutil.parser

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
n_days = 100
df = pd.DataFrame()
df['day'] = range(n_days)
df['sick'] = 0.0
starting_population = 6e6
df['population_left'] = starting_population #Susceptible population left
contagious_days=14
start = dateutil.parser.parse("2020-02-24T00:00:00Z")
df['cured'] = 0. # Cured
df['dead'] = 0. # Deaths per day
df['bodycount'] = 0. # Minimization goal
df['sporadic_additions'] = 0. #Sporadic additions from the news
df['ic'] = 0. # People in Intensive Care
df['no_ic'] = 0. # People left witout IC 
df['get_sick'] = 0. # People starting to show symptoms / getting sick
df.sick[0] = 1.0 # First patient

# Changes in R0 per week
df['r0'] = 4.2 # Starting with a lot of 'imported' cases
df.loc[df['day'] >3*7, 'r0'] = 3.2
df.loc[df['day'] >4*7, 'r0'] = 2.5 # Estimated infection rate
df.loc[df['day'] >5*7, 'r0'] = 2.5
df.loc[df['day'] >6*7, 'r0'] = 2.0
df.loc[df['day'] >7*7, 'r0'] = 1.5
df.loc[df['day'] >8*7, 'r0'] = 1.3
df.loc[df['day'] >10*7, 'r0'] = 0.9 # 
#df.loc[df['day'] >11*7, 'r0'] = 1.1
#df.loc[df['day'] >12*7, 'r0'] = 0.9
#df.loc[df['day'] >15*7, 'r0'] = 0.4

# Add from the News
df.sporadic_additions[17] = 17

#How many days the disease is contagious
contagious_days=14

#Probabilities for death under different conditions

death_prob_per_day = 0.037/contagious_days
death_prob_ic = 0.15/contagious_days
death_prob_no_care = 0.3/contagious_days

ic_need = 0.06
ic_capacity=1000.

for i in range(n_days-15):
    if df.sick[i]*ic_need > ic_capacity:
        ic_nocare_patients = df.sick[i]*ic_need - ic_capacity
        ic_patients = ic_capacity
    else:
        ic_nocare_patients = 0
        ic_patients = df.sick[i]*ic_need
    df.dead[i] = (death_prob_per_day*(df.sick[i] - ic_nocare_patients - ic_patients)
                  + death_prob_no_care * ic_nocare_patients
                  + death_prob_ic * ic_patients)
    df.ic[i] = ic_patients
    df.no_ic[i] = ic_nocare_patients
    
    df.bodycount[i+1] = df.bodycount[i] + abs(df.dead[i])

    get_sick = ( df.population_left[i] * df.sick[i] * (df.r0[i]/contagious_days) / starting_population
                 + df.sporadic_additions[i] 
               )
    if get_sick < 0:
        get_sick = 0
        
    df.get_sick[i] = get_sick
    df.cured[i+14] = get_sick - df.dead[i]
    
    df.sick[i+1] = (df.sick[i]  - df.cured[i] ) + get_sick - df.dead[i]
    
    df.population_left[i+1] = df.population_left[i] - df.cured[i] - df.dead[i] - get_sick
    
    if df.sick[i+1] < 0:
        df.sick[i+1] = 0.0
        #df.contagious[i] = 0
    if df.population_left[i] < 0:
        df.population_left[i] = 0
    #print((1+(df.contagious[i]-1.0)/14.0))

df.ts = pd.to_datetime(start.timestamp() + df['day']*86400, unit='s')
data=[
      go.Scatter(x=df.ts, y=df.sick, name="Sick"),
      #go.Scatter(x=df.ts, y=df.bodycount, name="Dead"),
      #go.Scatter(x=df.ts, y=df.ic, name="IC"),
      #go.Scatter(x=df.ts, y=df.no_ic, name="No IC"),
      #go.Scatter(x=df.ts, y=df.cured, name="Cured"),
      go.Scatter(x=df.ts, y=df.get_sick, name="Get sick"),
      #go.Scatter(x=df.ts, y=df.population_left, name="Susceptible"),
      go.Scatter(x=df.ts, y=df.r0, name="R0"),
      go.Scatter(x=df.ts, y=df.day, name="Day#"),

]
layout = go.Layout(
    title='Covid19 SIR+IC+D-model, Finland',
    xaxis=dict(
        title='day',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        type='log',
        title='people',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        ),
        #range = [0,6.5],
        #range = [0,60000],

    )
)
iplot({ 'data': data, "layout": layout }) 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:81: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

